### Theory About Reproducibility
Reproducing or rather producing the quantized model is the deadliest pain the ass I have felt in a while. It was bad.

Keeping the technicalities about SELECT_OPS and Flex Delegate etc aside, just confirm your setup as follow. I have already talked about using virtual environment as kernel in diskloader.ipynb

Final setup
- OS: Ubuntu Server  
- Python: 3.12  
- TensorFlow: 2.18.0 (regular, CPU-only)  
  - Do NOT use `tensorflow[and-cuda]` (conversion fails).  
  - Regular `tensorflow==2.18.0` works fine with Python 3.12.
    ```bash
    pip uninstall -y tensorflow[and-cuda]
    pip install tensorflow==2.18.*
    ````

### Prerequisites (uncomment to run)

#### Installations

In [1]:
# Assuming virtual environment .tens with python 3.12 is being used
%pip uninstall -y tensorflow[and-cuda]
%pip install tensorflow==2.18.* tqdm

Found existing installation: tensorflow 2.18.1
Uninstalling tensorflow-2.18.1:
  Successfully uninstalled tensorflow-2.18.1
Note: you may need to restart the kernel to use updated packages.
  Using cached tensorflow-2.18.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
Using cached tensorflow-2.18.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (615.6 MB)
Note: you may need to restart the kernel to use updated packages.


#### Data Preprocessing
- No need to re-run if already ran in `diskloader.ipynb`

In [2]:
# # Step 1: Download QuickDraw dataset
# %python download_data.py

# # Step 2: Combine stroke data across different partitions (train, val, test).
# # The original data (e.g., airplanes.npz) is split into these partitions, 
# # which reduces the number of data points. We merge all partitions for a complete dataset.
# %python combine_strokes.py

# # Step 3: Convert each `.npy` and `.npz` file into individual sample files.
# # For example, `airplanes.npy` contains (N, 784), where N is the number of samples 
# # and each sample is a flattened 28x28 image (784 features). We split this into
# # individual `.npy` files like `airplanes/000001.npy`, `airplanes/000002.npy`, etc.
# # This allows us to load specific slices of data (e.g., airplanes[3012:33012]) 
# # without having to load the entire file, reducing memory usage and avoiding memory eviction issues.
# %python convert_to_individual_npys.py

#### Imports

In [3]:
import json
import math
import os
from glob import glob

import numpy as np
import tensorflow as tf
from tqdm import tqdm

2025-08-24 16:59:03.655685: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756054743.668878  352251 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756054743.672396  352251 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Using the same dataloader pipeline as used in training

### Global Variables

In [4]:
# Files
LABELS_JSON    = "/home/priyanka/doodle-vision/training/dataset/label_map.json"
PROCESSED_DATA_DIR = "/home/priyanka/doodle-vision/training/dataset/processed"

# Data props
NUM_CLASSES = 345

# Training
SAMPLES_PER_CLASS = 30_000
SPLIT_RATIOS = (0.8, 0.1, 0.1) # train, val, test
BATCH_SIZE = 512

In [5]:
with open(LABELS_JSON, 'r') as f:
    LABEL_MAP = json.load(f)
    
REV_LABEL_MAP = {v: k for k, v in LABEL_MAP.items()}

### Disk Data Loader for Original Data

In [6]:
# Using the individual + in-memory mode for tflite inference
images = []
strokes = []

# Always list of integers
labels = []

for cls, label in tqdm(LABEL_MAP.items(), desc="Building dataset"):
    image_files = sorted(
        glob(os.path.join(PROCESSED_DATA_DIR, "images", cls, "*.npy"))
    )
    stroke_files = sorted(
        glob(os.path.join(PROCESSED_DATA_DIR, "strokes", cls, "*.npy"))
    )

    N = min(len(image_files), len(stroke_files), SAMPLES_PER_CLASS)

    for i in range(N):
        image_path = image_files[i]
        stroke_path = stroke_files[i]

        images.append(np.load(image_path))
        strokes.append(np.load(stroke_path))
        labels.append(label)

print(f"[In-Memory Mode] Loaded {len(labels):,} samples.")

Building dataset: 100%|███████████████████████████████████████████████████████████████| 345/345 [18:50<00:00,  3.28s/it]

[In-Memory Mode] Loaded 10,350,000 samples.


In [7]:
def data_generator(images, strokes, labels):
    for image, stroke, label in zip(images, strokes, labels):
        yield (
            image,
            stroke,
            tf.one_hot(label, depth=NUM_CLASSES),
        )

In [8]:
def format_sample(img, stroke, label):
    return {"stroke_input": stroke, "image_input": img}, label

output_signature = (
    tf.TensorSpec(shape=(28, 28, 1), dtype=tf.float32),
    tf.TensorSpec(shape=(130, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(NUM_CLASSES,), dtype=tf.int32),
)

### Train, Val, and Test Splits

In [9]:
# Shuffle once and split
total = len(labels)
indices = np.arange(total)
np.random.seed(42)
np.random.shuffle(indices)

train_end = int(SPLIT_RATIOS[0] * total)
val_end = train_end + int(SPLIT_RATIOS[1] * total)

train_idx = indices[:train_end]
val_idx = indices[train_end:val_end]
test_idx = indices[val_end:]

# Train
train_images =  [images[i] for i in train_idx]
train_strokes = [strokes[i] for i in train_idx]
train_labels =  [labels[i] for i in train_idx]

# Val
val_images =  [images[i] for i in val_idx]
val_strokes = [strokes[i] for i in val_idx]
val_labels =  [labels[i] for i in val_idx]

# Test
test_images =  [images[i] for i in test_idx]
test_strokes = [strokes[i] for i in test_idx]
test_labels =  [labels[i] for i in test_idx]

In [10]:
def build_dataset(images, strokes, labels, is_shuffle=False):
    def gen():
        return data_generator(images, strokes, labels)

    ds = tf.data.Dataset.from_generator(
        gen,
        output_signature=output_signature
    )

    if is_shuffle:
        ds = ds.shuffle(BATCH_SIZE * 10)

    ds = ds.repeat()
    ds = ds.map(format_sample, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE)
    return ds.prefetch(tf.data.AUTOTUNE)

In [11]:
test_ds = build_dataset(test_images, test_strokes, test_labels).take(math.ceil(len(test_labels) / BATCH_SIZE))

W0000 00:00:1756055885.711058  352251 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Quantizing the Model

In [12]:
# Load the original model
model = tf.keras.models.load_model('/home/priyanka/doodle-vision/training/models/hybrid_345_classes_30000_examples.keras')

In [13]:
original_model_size = os.path.getsize(
    '/home/priyanka/doodle-vision/training/models/hybrid_345_classes_30000_examples.keras'
)
print(f"Original Keras Model Size: {original_model_size / (1024 * 1024):.2f} MB")


weights_only_path = '/home/priyanka/doodle-vision/training/models/hybrid_345_classes_30000_examples.weights.h5'
model.save_weights(weights_only_path)

weights_size_bytes = os.path.getsize(weights_only_path)
print(f"Weights-Only Model Size: {weights_size_bytes / (1024 * 1024):.2f} MB")

Original Keras Model Size: 14.86 MB
Weights-Only Model Size: 14.84 MB


In [14]:
print("Precision of the model's layers:")
for layer in model.layers:
    if hasattr(layer, 'kernel'):
        print(f"Layer '{layer.name}': {layer.kernel.dtype}")

Precision of the model's layers:
Layer 'conv2d': float32
Layer 'conv2d_1': float32
Layer 'conv2d_2': float32
Layer 'conv2d_3': float32
Layer 'dense': float32
Layer 'dense_1': float32
Layer 'dense_2': float32
Layer 'dense_3': float32


### 1. Dynamic Range Quantization
This is the default quantization method. It quantizes the model's weights to 8-bit precision during conversion. This reduces model size but keeps activations in floating-point, offering a good balance between size reduction and minimal accuracy loss.

In [15]:
def dynamic_range_quantization(keras_model, save_path):
    converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()
    with open(save_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Dynamic range quantized model saved to {save_path}")
    return os.path.getsize(save_path)

dynamic_range_size = dynamic_range_quantization(model, 'best_model_345_c_30000_ex_dynamic_quant.tflite')

INFO:tensorflow:Assets written to: /tmp/tmpl5q5rvfe/assets


INFO:tensorflow:Assets written to: /tmp/tmpl5q5rvfe/assets


Saved artifact at '/tmp/tmpl5q5rvfe'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 130, 3), dtype=tf.float32, name='stroke_input'), TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='image_input')]
Output Type:
  TensorSpec(shape=(None, 345), dtype=tf.float32, name=None)
Captures:
  132505096533968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096535888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096535696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096534160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096533392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096535312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096535120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096528208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505121395856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505

W0000 00:00:1756055889.713178  352251 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1756055889.713203  352251 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1756055889.735993  352251 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


### 2. Float16 Quantization

This method converts model weights to 16-bit floating-point numbers. It offers a 50% reduction in model size compared to the original 32-bit float model, with almost no loss in accuracy. This is a good middle ground if dynamic range or integer quantization leads to unacceptable accuracy drops.

In [16]:
def float16_quantization(keras_model, save_path):
    converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    tflite_model = converter.convert()
    with open(save_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Float16 quantized model saved to {save_path}")
    return os.path.getsize(save_path)

float16_size = float16_quantization(model, 'best_model_345_c_30000_ex_fp16_quant.tflite')

INFO:tensorflow:Assets written to: /tmp/tmpurhncox0/assets


INFO:tensorflow:Assets written to: /tmp/tmpurhncox0/assets


Saved artifact at '/tmp/tmpurhncox0'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 130, 3), dtype=tf.float32, name='stroke_input'), TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='image_input')]
Output Type:
  TensorSpec(shape=(None, 345), dtype=tf.float32, name=None)
Captures:
  132505096533968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096535888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096535696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096534160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096533392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096535312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096535120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505096528208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505121395856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132505

W0000 00:00:1756055891.391189  352251 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1756055891.391202  352251 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


### Evaluation

In [17]:
def evaluate_tflite_model(model_path, test_dataset):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Find the indices of the 'image_input' and 'stroke_input'
    image_idx = [i for i, d in enumerate(input_details) if "image_input" in d["name"]][0]
    stroke_idx = [i for i, d in enumerate(input_details) if "stroke_input" in d["name"]][0]
    output_idx = 0

    correct = 0
    total = 0
    pbar = tqdm(test_dataset, desc=f"Evaluating {os.path.basename(model_path)}", dynamic_ncols=True)
    
    for batch in pbar:
        inputs, labels = batch
        images = inputs["image_input"].numpy()
        strokes = inputs["stroke_input"].numpy()
        labels = labels.numpy()

        for i in range(len(images)):
            # Set input tensors
            interpreter.set_tensor(input_details[image_idx]['index'], images[i:i+1].astype(np.float32))
            interpreter.set_tensor(input_details[stroke_idx]['index'], strokes[i:i+1].astype(np.float32))

            # Run inference
            interpreter.invoke()

            # Get predictions
            preds = interpreter.get_tensor(output_details[output_idx]['index'])
            pred_class = np.argmax(preds, axis=1)[0]
            true_class = np.argmax(labels[i])

            if pred_class == true_class:
                correct += 1
            total += 1

        pbar.set_postfix(acc=f"{correct/total:.4f}")

    return correct, total

In [18]:
# Evaluate Float16 Quantized Model
correct_f16, total_f16 = evaluate_tflite_model('best_model_345_c_30000_ex_fp16_quant.tflite', test_ds)
accuracy_f16 = correct_f16 / total_f16
print(f"Float16 Quantization: Size = {float16_size / (1024 * 1024):.2f} MB, Accuracy = {accuracy_f16:.4f}")

/home/priyanka/doodle-vision/.tens/lib/python3.12/site-packages/tensorflow/lite/python/interpreter.py:451: UserWarning:     Warning: Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite delegate for select TF ops.
W0000 00:00:1756055891.904112  352251 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO: TfLiteFlexDelegate delegate: 8 nodes delegated out of 82 nodes with 4 partitions.

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Evaluating best_model_345_c_30000_ex_fp16_quant.tflite: 100%|█████████| 2022/2022 [3:1

Float16 Quantization: Size = 2.52 MB, Accuracy = 0.4744


In [19]:
# Evaluate Dynamic Range Quantized Model
correct_dr, total_dr = evaluate_tflite_model('best_model_345_c_30000_ex_dynamic_quant.tflite', test_ds)
accuracy_dr = correct_dr / total_dr


W0000 00:00:1756067700.994469  352251 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
IOPub message rate exceeded.30000_ex_dynamic_quant.tflite:  74%|████▍ | 1499/2022 [1:29:25<31:10,  3.58s/it, acc=0.8917]
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [20]:
print(f"Dynamic Range Quantization: Size = {dynamic_range_size / (1024 * 1024):.2f} MB, Accuracy = {accuracy_dr:.4f}")

Dynamic Range Quantization: Size = 1.37 MB, Accuracy = 0.8916
